In [1]:
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1

In [2]:
import os

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from ibm_watson_openscale.base_classes.watson_open_scale_v2 import WMLCredentialsCP4D
from ibm_cloud_sdk_core.authenticators import BearerTokenAuthenticator
from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

# Create Dummy Headless ML Provider in OpenScale

This is used when there is no need for OpenScale to communicate with the deployment (e.g., send scoring request to the endpoint).

## 1. Setup

In [3]:
# Initialize connection to WOS client
authenticator = BearerTokenAuthenticator(
        bearer_token=os.environ['USER_ACCESS_TOKEN']
    )

wos_client = APIClient(authenticator=authenticator, 
                       service_url=os.environ['RUNTIME_ENV_APSX_URL'])
wos_client.version

'3.0.14'

## 2. Identify data mart ID

In [4]:
wos_client.data_marts.show()

AIOSFASTPATHICP-00000000-0000-0000-0000-000000000000,Data Mart created by OpenScale ExpressPath,False,active,2022-01-13 06:19:21.327000+00:00,00000000-0000-0000-0000-000000000000


In [5]:
data_marts = wos_client.data_marts.list().result.data_marts
DATA_MART_ID = data_marts[0].metadata.id
DATA_MART_ID 
print("Data Mart ID: {}".format(DATA_MART_ID))

Data Mart ID: 00000000-0000-0000-0000-000000000000


## 3. Create headless service provider

In [6]:
# Name the service provider
SERVICE_PROVIDER_NAME = "OpenScale Headless Service Provider"
SERVICE_PROVIDER_DESCRIPTION = ""

In [7]:
existing_providers_dict = wos_client.service_providers.list().get_result().to_dict()['service_providers']
existing_providers = [sp['entity']['name'] for sp in existing_providers_dict]

if not SERVICE_PROVIDER_NAME in existing_providers:
    added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.CUSTOM_MACHINE_LEARNING,
        operational_space_id = "production",
        credentials=WMLCredentialsCP4D(),
        background_mode=False
     ).result
    
    SERVICE_PROVIDER_ID = added_service_provider_result.metadata.id
    print(f"Service provider ID: {SERVICE_PROVIDER_ID}")
else:
    SERVICE_PROVIDER_ID = next((sp['metadata']['id'] for sp in existing_providers_dict \
                                if sp['entity']['name'] == SERVICE_PROVIDER_NAME))
    print(f"Service provider ID: {SERVICE_PROVIDER_ID}")

Service provider ID: 1059fa1d-7dfc-4d7b-8e40-ae8aef72afe1
